# Experimenting with Prophet Data Preparation

This notebook demonstrates how to download and transform stock data for use with Prophet.

In [ ]:
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go

def download_stock_data(ticker_symbol):
    """
    Download stock data with error handling and validation
    """
    try:
        # Create ticker object and get history
        yticker = yf.Ticker(ticker_symbol)
        df = yticker.history(period='max')
        
        if df.empty:
            raise ValueError(f"No data downloaded for {ticker_symbol}")
            
        print(f"Downloaded {len(df)} days of {ticker_symbol} data")
        
        # Basic validation
        required_columns = ['Open', 'Close', 'Volume']
        missing_cols = [col for col in required_columns if col not in df.columns]
        if missing_cols:
            raise ValueError(f"Missing required columns: {missing_cols}")
            
        return df
    
    except Exception as e:
        print(f"Error downloading {ticker_symbol}: {str(e)}")
        return None

def transform_stock_data(df, ticker):
    """
    Transform stock data for Prophet model
    """
    if df is None:
        return None
    
    # Reset index to make the date a column
    df = df.reset_index()
    
    # Prophet requires columns named 'ds' (date) and 'y' (target)
    prophet_df = pd.DataFrame()
    prophet_df['ds'] = pd.to_datetime(df['Date']).dt.tz_localize(None)  # Remove timezone
    prophet_df['y'] = df['Close'].astype(float)    # Ensure float
    
    # Add additional features as regressors
    prophet_df['volume'] = df['Volume'].astype(float)
    prophet_df['open'] = df['Open'].astype(float)
    
    # Calculate returns
    prophet_df['returns'] = prophet_df['y'].pct_change()
    
    # Calculate moving averages
    prophet_df['ma20'] = prophet_df['y'].rolling(window=20).mean()
    prophet_df['ma50'] = prophet_df['y'].rolling(window=50).mean()
    
    # Calculate volatility (20-day rolling standard deviation)
    prophet_df['volatility'] = prophet_df['y'].rolling(window=20).std()
    
    # Add day of week as a feature
    prophet_df['day_of_week'] = prophet_df['ds'].dt.dayofweek
    
    # Fill any missing values
    prophet_df = prophet_df.ffill()
    
    return prophet_df

## Download Stock Data

In [ ]:
# Download data for QQQ
ticker = "QQQ"
df = download_stock_data(ticker)

print("\nRaw data sample:")
df.head()

## Transform Data for Prophet

Prophet requires specific column names (read docs here: https://facebook.github.io/prophet/docs/quick_start.html):
- 'ds': date column (should be without timezone)
- 'y': target variable

We'll also add additional features as regressors.

In [ ]:
# Transform data into Prophet format
prophet_df = transform_stock_data(df, ticker)

print("\nTransformed data sample (Prophet format):")
prophet_df.head()

## Available Features

Let's look at all the features we've created for Prophet:

In [ ]:
print("Available features:")
prophet_df.columns.tolist()

# Predict with Prophet

In [ ]:
m = Prophet()
m.fit(prophet_df)



## Visualize the Data

In [ ]:
# Create interactive plot
fig = go.Figure()

# Plot price
fig.add_trace(go.Scatter(
    x=prophet_df['ds'],
    y=prophet_df['y'],
    name='Price',
    line=dict(color='blue')
))

# Add moving averages
fig.add_trace(go.Scatter(
    x=prophet_df['ds'],
    y=prophet_df['ma20'],
    name='MA20',
    line=dict(color='orange', dash='dash')
))

fig.add_trace(go.Scatter(
    x=prophet_df['ds'],
    y=prophet_df['ma50'],
    name='MA50',
    line=dict(color='red', dash='dash')
))

fig.update_layout(
    title=f'{ticker} Stock Price with Moving Averages',
    yaxis_title='Price',
    template='presentation'
)
fig.show()